# LLM From Scratch

This is a notebook I'm using to re-create the GPT-2 style architecture from the book "Build a Large Language Model (From Scratch)."
I'm trying to do as much as possible from memory, other than having some notes on what classes and methods to implement.

**Required classes:**
1. LayerNorm
2. GELU
3. FeedForward
4. MultiHeadAttention
5. TransformerBlock
6. GPT_CONFIG_124M
7. GPTModel

In [1]:
# Import torch and nn.Module for class definitions
import torch
import torch.nn as nn

## 1. LayerNorm

This class is responsible for layer normalization, which takes place _multiple times_ in the GPT architecture.
Its purpose is to keep gradient magnitudes within a certain range, to avoid the problems of vanishing gradients and exploding gradients.
The concrete goal is to adjust the outputs to have a mean of zero and a variance of one.

To accomplish this, we need two values:
- the mean: $\mu = \frac{(x_1 + x_2 + ... + x_n)}{n}$
- the variance: $v = \frac{(x_1 + \mu) + (x_2 + \mu) + ... + (x_n + \mu)}{n} + \epsilon$

The normalized vector is then: $[\frac{(x_1 - µ)}{\sqrt{v}}, \frac{(x_2 - µ)}{\sqrt{v}}, ..., \frac{(x_n - µ)}{\sqrt{v}}]$

NOTE: we're dividing by both n and $\sqrt{v}$ and we need to make sure we never divide by zero. We know that n (the embedding dimension) will never be zero, but the variance could be. For that reason, we add a miniscule value epsilon to the variance.

In [2]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim: int):
        super().__init__()
        self.emb_dim = emb_dim
        self.epsilon = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        mean = x.mean(dim=-1, keepdim=True)
        variance = x.var(dim=-1, keepdim=True, unbiased=False) + self.epsilon
        norm = (x - mean) / torch.sqrt(variance)
        return self.scale * norm + self.shift

## 2. GELU

GELU, or Gaussian Error Linear Unit, is the activation function we'll be using. It's similar to RELU, but it's differentiable everywhere (even at zero, where RELU has a sharp corner discontinuity). GELU is also slightly negative between -2 and 0, rather than flatly zero like RELU. This provides a richer range of values for the network to train on.

Calculating the GELU for real would take us out of closed-form math, so we'll use a very close approximation here instead.

In [3]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
        ))

## 3. GPT_CONFIG_124M
The configuration paramters for our GPT-2 implementation. These come directly from the book.

In [4]:
from typing import TypedDict

class GPTConfigDict(TypedDict):
    vocab_size: int        # the number of tokens in the vocabulary
    context_length: int    # the maximum number of token vectors to consider at once
    emb_dim: int           # the width of the token vectors
    n_heads: int           # the number of heads to use for multi-head attention
    n_layers: int          # the number of transformer layers to use
    drop_rate: float       # the dropout percentage rate
    qkv_bias: bool         # whether to use the bias setting for the KQV matrices.

GPT_CONFIG_124M = {
    "vocab_size": 50257,
    "context_length": 1024,
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.1,
    "qkv_bias": False,
}

## 4. FeedForward

The FeedForward network (or multi-layer perceptron) is the fundamental neural network used in the GPT model.
It expands the number of outputs in a hidden layer before shrinking back down to the original size for the output.
This allows the network to explore a richer space, while preserving the input and output dimensions to keep the overall architecture simple.

In this example, we'll expand the dimensions by a factor of 4 for the internal layer. I would normally say that should be configurable, but the book just has it fixed at 4. Anyway, that means that our 768 parameters will expand to 3,072, then shrink back down to 768 for output.

In [5]:
class FeedForward(nn.Module):
    def __init__(self, cfg: GPTConfigDict): 
        super().__init__()
        expansion_factor = 4
        dim_external = cfg["emb_dim"]
        dim_internal = expansion_factor * dim_external
        self.layers = nn.Sequential(
            nn.Linear(dim_external, dim_internal),
            GELU(),
            nn.Linear(dim_internal, dim_external),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.layers(x)

## 5. MultiHeadAttention

This is the heart of what makes GPT different to earlier language models. The attention mechanism tweaks context vectors in response to earlier tokens in the sequence, shifting their "meaning" to become much richer and more specific than a single word could be.

### Motivating Examples

For example, take the sentence "the cat sat on the mat because it was warm." The word "it" has one particular vector embedding in the vocabulary, which might relate loosely to concepts like "noun" and "non-human." That's not enough to capture the meaning of "it" in this sentence, where it most likely refers to "mat." Attention allows the system to change the vector for the "it" token to resemble the vector for "mat," clarifying its meaning in the context of the sentence.

That's about the simplest possible example, but in reality each token is pushed and pulled in much more subtle ways by many more tokens in the sequence, so that by the end it somehow represents the meaning of the entire sequence of text. Ultimately, the attention-modulated vector of the final token in the sequence is _the only input needed_ to predict the next token. That's pretty wild.

For a more contrived example of what this means, take another example sequence: "This gritty, romantic, windswept, ornate, melancholic city is none other than". The word "than" has nothing to do with any particular city or place, but by the time its vector is modulated by this long series of words preceding it, it will be something that appears close (in embedding space) to cities like Lisbon and Istanbul. Indeed, those are the two most likely predictions for the final word in the sequence from GPT-3.

### Implementation

Multi-head attention was first described in "Attention is All You Need" (2017), in sections 3.2.1 (scaled dot-product attention) and 3.2.2 (extending to multiple heads). I'll be using that paper as a reference for the following two sections.

#### Scaled Dot-Product Attention

Each attention head is an instance of something called "scaled dot-product attention," which is given by:

$\text{Attention}(Q, K, V) = \text{softmax}(\frac{QK^T}{\sqrt{d_k}})V$

That is, the attention weights given matrices K, Q, and V are the result of applying softmax to the product of Q times K-transpose over the square root of the embedding size of K, all multiplied by V.

I'll try to break that down a bit more:
- Q, K, and V are trainable matrix parameters with the same dimensions as the token embedding vectors. They are short for Query, Key, and Value.
  - I think of the Query parameter as representing what a token is "looking for" to know if another token is worth attending to.
  - To continue that metaphor, the Key parameter is what other tokens "look like" to the Query.
  - The Value is the real identity of the tokens that are found, their deeper reality beneath the appearance presented by the Key.
  - To sum up, a token's Query is used to examine every other token's Key to see if it's a good match. If it is, we use that token's Value in attention weight.
- Multiplying Q by the transpose of K gives us the dot product of every Query row against every Key row. In other words, it tells us how aligned every Query is with every Key.
- We scale that by the inverse square root of the Key dimensions to counteract a known issue with dot-product attention: "for large values of d_k, the dot products grow large in magnitude, pushing the softmax function into regions where it has extremely small gradients." ("Attention is All You Need," p. 4). In other words, the dot product of two rows is going to tend to get larger the more columns you have, and these large values make it hard for training to adjust weights effectively. Scaling by the square root of the number of columns helps to solve this.
- Applying softmax turns these scaled dot products into weights.
- Multiplying by V translates the weights by Key into weights by Value.

Note: it's not described in detail in the paper, but there's an important step carried out here called masking. Essentially, we only want Queries to find Keys that _precede_ them in the sequence. We accomplish this by zeroing out values above the main diagonal. To make sure that these values are zero _after_ softmax, we first set them to minus-infinity.

#### Multi-Head Attention

In single-headed dot-product attention, Q, K, and V all have the same dimensions as the input and output embeddings. To use multiple heads, we divide the width of each parameter by the number of heads and concatenate them together. This results in the same overall dimensions, but with different sets of columns relating to different Value vectors:

$\text{MultiHead}(Q, K, V) = \text{Concat}(head_1, ..., head_h)W^O$

$\text{ where } head_i = \text{Attention}(Q_iW_i^Q, K_iW_i^K, V_iW_i^V)$

$\text{ where } W_i^Q \in \Reals^{d_{model} \times d_k}$, $W_i^K \in \Reals^{d_{model} \times d_k}$, $ W_i^V \in \Reals^{d_{model} \times d_v}$, $W_i^O \in \Reals^{hd_{model} \times d_{model}}$


In [15]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in: int, d_out: int, context_length: int, dropout: float, num_heads: int, qkv_bias: bool=False):
        super().__init__()
        if d_out % num_heads != 0:
            raise ValueError("The number of heads must evenly divide d_out.")
        self.d_in = d_in
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_width = d_out // num_heads
        self.qkv_bias = qkv_bias

        # construct the weights for Q, K, and V.
        # these will be registered as trainable parameters automatically.
        self.w_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        # and the output projection, also trainable.
        self.w_out = nn.Linear(d_out, d_out)
        
        # and the dropout layer. not trainable, just drops random values
        # to zero with a probability determined by the dropout parameter
        self.dropout = nn.Dropout(dropout)

        # and the mask, which prevents each token from "seeing" later ones
        mask = torch.triu( # an upper triangular matrix
            torch.ones(context_length, context_length), # consisting of ones
            diagonal=1, # starting one row above the diagonal, leaving the diagonal itself as zeroes.
        )
        self.register_buffer("mask", mask) # register this tensor as non-trainable, but keep it on the same device

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        batch, num_tokens, d_in = x.shape
        queries = self.w_query(x)
        keys = self.w_key(x)
        values = self.w_value(x)

        # Split the last dimension of the tensors into multiple heads
        q_heads = queries.view(batch, num_tokens, self.num_heads, self.head_width)
        k_heads = keys.view(batch, num_tokens, self.num_heads, self.head_width)
        v_heads = values.view(batch, num_tokens, self.num_heads, self.head_width)

        #                                  [  0  ,     1     ,    2     ,      3    ]
        # {q,k,v}_heads now have the shape [batch, num_tokens, num_heads, head_width],
        # but we want them to be:          [batch, num_heads, num_tokens, head_width]
        q_heads = q_heads.transpose(1, 2)
        k_heads = k_heads.transpose(1, 2)
        v_heads = v_heads.transpose(1, 2)

        # now we need to calculate the raw dot-product attention scores between Q and K^T,
        # where K^T has the shape [batch, num_heads, head_width, num_tokens].
        # that gives attention_scores the shape [batch, num_heads, num_tokens, num_tokens]
        attention_scores = q_heads @ k_heads.transpose(2, 3)
        # and apply the causal mask
        mask = self.mask[:num_tokens, :num_tokens]
        attention_scores = attention_scores.masked_fill(mask == 1, float('-inf'))

        # and we construct the weights using softmax on the scaled final dimension
        attention_weights = torch.softmax(attention_scores / self.head_width**0.5, dim=-1)
        # and apply dropout
        attention_weights = self.dropout(attention_weights)

        #                                 [  0  ,     1    ,     2     ,     3     ]
        # attention_weights has the shape [batch, num_heads, num_tokens, num_tokens]
        # v_heads has the shape:          [batch, num_heads, num_tokens, head_width]
        # if we multiply them, we get:    [batch, num_heads, num_tokens, head_width]
        # but in the end, we want:        [batch, num_tokens, d_out]
        context = attention_weights @ v_heads # [batch, num_heads, num_tokens, head_width]

        # so we need to first transpose and get [batch, num_tokens, num_heads, head_width]
        context = context.transpose(1, 2)
        # and then concatenate the last two dimensions together to get d_out
        context = context.contiguous().view(batch, num_tokens, self.d_out)
        # and multiply by the output projection
        return self.w_out(context)

## 6. TransformerBlock
To be implemented.

In [7]:
class TransformerBlock:
    def __init__(self):
        pass

## 7. GPTModel
Top-level GPT-2 model class.

In [8]:
class GPTModel(nn.Module):
    """
    Top-level GPT-2 model.
    """
    # cfg: Cfg -> GPTModel
    def __init__(self, cfg: GPTConfigDict):
        """Initialize model with config."""
        super().__init__()
        pass

    # in_idx: torch.Tensor -> logits: torch.Tensor
    def forward(self, in_idx: torch.Tensor) -> torch.Tensor:
        """Forward pass: input indices to logits."""
        pass